In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

### Loading dataset and making it iteratable

In [2]:
train_data = torchvision.datasets.MNIST(root='data',transform=transforms.ToTensor(),train = True)
test_data = torchvision.datasets.MNIST(root='data',transform=transforms.ToTensor(),train = False)

In [3]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=100,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,batch_size=100,shuffle=True)

### Creating a model class

In [10]:
class RNN(nn.Module):
    
    def __init__(self,input_size,output_size,hidden_size,num_layers):
        
        super(RNN,self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size=input_size,hidden_size=hidden_size,num_layers=self.num_layers,batch_first=True,nonlinearity='relu')
        
        self.fc1 = nn.Linear(hidden_size,output_size)
        
    
    def forward(self,X):
        
        h0 = torch.randn(self.num_layers,X.size(0),self.hidden_size)
        out,h = self.rnn(X,h0)
        out = out[:,-1,:]
        out = self.fc1(out)
        
        return out

In [11]:
model = RNN(input_size=28,output_size=10,hidden_size=100,num_layers=1)

### Creating Loss function and optimizer

In [12]:
loss_func = nn.CrossEntropyLoss()
adam = torch.optim.SGD(model.parameters(),lr=0.1)

In [13]:
seq_lenght = 28
input_size = 28

### Trainning

In [15]:
for epoc in range(5):
    
    for i,(images,labels) in enumerate(train_loader):

        images = images.view(-1,seq_lenght,input_size)

        out = model.forward(images)

        loss = loss_func(out,labels)

        adam.zero_grad()
        loss.backward()
        adam.step()

        if i %100 == 0:

            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoc+1, 5, i+1, len(train_loader), loss.item()))


Epoch [1/5], Step [1/600], Loss: 2.3009
Epoch [1/5], Step [101/600], Loss: 2.2958
Epoch [1/5], Step [201/600], Loss: 2.2556
Epoch [1/5], Step [301/600], Loss: 1.7866
Epoch [1/5], Step [401/600], Loss: 1.7599
Epoch [1/5], Step [501/600], Loss: 1.3238
Epoch [2/5], Step [1/600], Loss: 1.3926
Epoch [2/5], Step [101/600], Loss: 1.2439
Epoch [2/5], Step [201/600], Loss: 0.8380
Epoch [2/5], Step [301/600], Loss: 1.2381
Epoch [2/5], Step [401/600], Loss: 0.7873
Epoch [2/5], Step [501/600], Loss: 0.5816
Epoch [3/5], Step [1/600], Loss: 0.6455
Epoch [3/5], Step [101/600], Loss: 0.8050
Epoch [3/5], Step [201/600], Loss: 0.5333
Epoch [3/5], Step [301/600], Loss: 0.4524
Epoch [3/5], Step [401/600], Loss: 0.4960
Epoch [3/5], Step [501/600], Loss: 0.3059
Epoch [4/5], Step [1/600], Loss: 0.5181
Epoch [4/5], Step [101/600], Loss: 0.2111
Epoch [4/5], Step [201/600], Loss: 0.3690
Epoch [4/5], Step [301/600], Loss: 0.5155
Epoch [4/5], Step [401/600], Loss: 0.3793
Epoch [4/5], Step [501/600], Loss: 1.6580


In [17]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1,seq_lenght,input_size)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 79.92 %
